# Alpaca data tick import & processing
*Note: I do not intend to use Alpaca data, as it only has IEX data. Also it has no delisted stocks etc. This is an exercise for myself such that I can easily manipulate data later on when I have a better data subscription. I will likely use [polygon](https://polygon.io/) for historical data and IBKR for real-time data. Even if I trade on Alpaca I will use the IBKR (5/month) live feed, because Alpaca streaming data only contains IEX, and I will not pay $99/month just to get real-time data. I hate that I do not have access to most US brokers, because all US brokers have free real-time data in contrast to Europe.*

This notebook downloads tick data and converts them into 1-minute quotes.

My public and private key are in ../data/alpaca/data/secret.txt

First, it downloads tick data and saves them to ../data/alpaca/raw/tick/{stock}/{date}.csv

Second it converts the tick data and saves the quotes to ../data/alpaca/processed/m1/quotes. The quotes contain the columns         <code>["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask", "tradeable"]</code>. The column 'tradable' is True if the [condition](https://alpaca.markets/docs/market-data/#uqdf) 'R' (regular trading) applies. 

Third, it deletes the tick data.

Finally, it merges the 1-minute quotes with the 1-minute OHLC to get quotebars.

In [2]:
from alpaca.data import StockHistoricalDataClient
from alpaca.data.requests import StockQuotesRequest
from datetime import datetime, time
from pytz import timezone

import pandas as pd
import numpy as np
import os

**Step 1: download NBBO tick data.**

Downloading SPY data for one day took 30 minutes. And this is only from the IEX exchange...

In [3]:
SYMBOL_LIST = ["TOP", "APE", "DEM"]
#TOP: multi-day halts, normale halts, listed second half 2022
#APE: very volatile, listed second half 2022
#DEM: has dividends, lot of missing data
START_DATE = datetime(2022, 1, 1) #datetime(2015, 12, 1) is the first available date from Alpaca
END_DATE = datetime(2023, 7, 21)
#END_DATE = datetime(2023, 2, 4, hour=20)  # in ET

Because it will take too much memory or take too long, we will send a request for every day between START_DATE end END_DATE. To get all trading days between these two days, we look at the trading days from SPY.

In [4]:
SPY_df = pd.read_csv(
        f"../data/alpaca/raw/d1/adjusted/SPY.csv",
        index_col="datetime",
        parse_dates=True,
    )
SPY_df.set_index(SPY_df.index.tz_convert("US/Eastern"), inplace=True) #Actually this isn't necessary, but we still do it for the sake of consistency
SPY_df.set_index(SPY_df.index.tz_localize(None), inplace=True)
dates = np.unique(pd.to_datetime(SPY_df.index).date)
trading_dates = dates[(dates >= START_DATE.date()) & (dates <= END_DATE.date())]
print(trading_dates[:5])
print(trading_dates[-5:])

[datetime.date(2022, 1, 3) datetime.date(2022, 1, 4)
 datetime.date(2022, 1, 5) datetime.date(2022, 1, 6)
 datetime.date(2022, 1, 7)]
[datetime.date(2023, 7, 17) datetime.date(2023, 7, 18)
 datetime.date(2023, 7, 19) datetime.date(2023, 7, 20)
 datetime.date(2023, 7, 21)]


In [ ]:
print(f"{datetime.utcnow().replace(microsecond=0)} | Starting download")
with open("../data/alpaca/secret.txt") as f:
    PUBLIC_KEY = next(f).strip()
    PRIVATE_KEY = next(f).strip()

stock_client = StockHistoricalDataClient(PUBLIC_KEY, PRIVATE_KEY)

for stock in SYMBOL_LIST:
    os.makedirs(f"../data/alpaca/raw/tick/{stock}", exist_ok=True) #Create folder if it does not exist
    for date in trading_dates:
        start_time = timezone("US/Eastern").localize(datetime.combine(date, time(hour=4, minute=0)))
        end_time = timezone("US/Eastern").localize(datetime.combine(date, time(hour=20, minute=0)))
        quote_request = StockQuotesRequest(
            symbol_or_symbols=stock, start=start_time, end=end_time
        )

        #Sometimes stock_client returns None. This can happen if the stock was not listed yet or if there are multi-day halts. In this case we should just skip the day.
        try:
            quotes = stock_client.get_stock_quotes(quote_request)
        except AttributeError as e:
            print(f"WARNING: caught AttributeError ({e}) while downloading {stock} at {date.strftime('%Y-%m-%d')}. Likely not listed or a multi-day halt.")
            continue
        
        quotes.df.to_csv(f"../data/alpaca/raw/tick/{stock}/{date.strftime('%Y-%m-%d')}.csv")
        print(f"{datetime.utcnow().replace(microsecond=0)} | Downloaded {stock} tick data for {date.strftime('%Y-%m-%d')}")

In [48]:
example = pd.read_csv( 
                f"../data/alpaca/raw/tick/DEM/2023-01-03.csv",
                usecols=[
                    "timestamp",
                    "bid_size",
                    "bid_price",
                    "ask_price",
                    "ask_size",
                    "conditions",
                ],
                index_col="timestamp",
                parse_dates=True,
                nrows=5
            )
example

,ask_price,ask_size,bid_price,bid_size,conditions
timestamp,,,,,
2023-01-03 12:00:00.039820+00:00,0.00,0.0,30.69,1.0,['R']
2023-01-03 12:00:00.045994+00:00,52.51,3.0,30.69,1.0,['R']
2023-01-03 12:00:00.077148+00:00,52.51,3.0,32.25,1.0,['R']
2023-01-03 12:00:04.917189+00:00,52.51,3.0,35.10,1.0,['R']
2023-01-03 12:00:12.684785+00:00,48.35,1.0,35.10,1.0,['R']


**Step 2: Retrieve NBBO data and convert to 1-minute quotes.**


We cannot just use resample and then ohlc, because of halts and other special trading conditions. Also we have to think about what OHLC means and what we want. To recap, the quotes contain the columns <code>["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask", "tradeable"]</code>. 

For the corresponding bid or ask in a minute timestamp at 10:00:00:

* close: we want the price at 10:00:59.99999..., which is the same as 10:01:00000... The quotes at this timestamp are the last quotes. Because if no new quotes come in, the last quotes stand. If the last available tick condition is not R ('regular trading'), the quote bar becomes untradable (and "tradable" = <code>False</code>). For example Y means a halt.
* open: the open has no use, it is always equal to the previous close. That is why we do not have open prices in the quotes. It is **incorrect** to use ohlc() if we do want a opening bid/ask, because the open takes the first value in the bin. Which introduces look-ahead bias. It should use the last available value.
* high: get highest value of regular trading (excluding non-regular trading). In the backtester high & lows will only be used to accurately simulate limit and stop orders.
* low: get lowest value of regular trading (excluding non-regular trading)

*Regular trading means no halts or weird conditions. It has nothing to do with pre- or post-market.*
* Step 1: Resample closes
* Step 2: Create a 'tradeable' column
* Step 3: Get the high and low bid/asks
* Step 4: Concatenate daily DataFrames

In [ ]:
# SYMBOL_LIST =

In [ ]:
print(f"{datetime.utcnow().replace(microsecond=0)} starting...")
for stock in SYMBOL_LIST:
    quote_dfs = [] #A collection of dataframes for every day
    for date in trading_dates:
        #Need to catch if does not exist
        try:
            tick_day_df = pd.read_csv( 
                f"../data/alpaca/raw/tick/{stock}/{date.strftime('%Y-%m-%d')}.csv",
                usecols=[
                    "timestamp",
                    "bid_size",
                    "bid_price",
                    "ask_price",
                    "ask_size",
                    "conditions",
                ],
                index_col="timestamp",
                parse_dates=True,
            )
        except FileNotFoundError:
            # Either we didn't download it, or stock was not listed yet (e.g. APE) or there was a multi-day halt (e.g. TOP).
            print(f"WARNING: There is no data for {stock} at {date.strftime('%Y-%m-%d')}.")
            continue

        tick_day_df.index.names = ["datetime"]
        
        # Convert to ET-naive as always (raw data is always in UTC-aware)
        tick_day_df.set_index(tick_day_df.index.tz_convert("US/Eastern"), inplace=True)
        tick_day_df.set_index(tick_day_df.index.tz_localize(None), inplace=True)

        #Step 1: To get the close quotes and last trade condition for m1, resample and take last value
        minute_df = tick_day_df.resample("1Min").last()
        minute_df.rename(
            columns={
                "bid_size": "close_bid_size",
                "bid_price": "close_bid",
                "ask_price": "close_ask",
                "ask_size": "close_ask_size",
            },
            inplace=True,
        )
        #Due to resampling it also creates rows for weekends and non-trading hours. Hence we need to shrink it
        all_minutes_in_day = []
        for hour in range(4, 20):
            for minute in range(0, 60):
                all_minutes_in_day.append(
                    datetime.combine(date, time(hour=hour, minute=minute))
                )
        all_minutes_in_day = np.array(all_minutes_in_day)
        assert len(all_minutes_in_day) == 16 * 60

        minute_df = minute_df.reindex(all_minutes_in_day)
        
        # Step 2: Create a 'tradeable' column which is False if there is no R in the conditions.
        #Warning: the values in the "conditions" are actually strings which look like lists. 
        # So we are checking if the letter R is in the string '[R]'. 
        # We also have to check whether the value is empty (thus not a string). 
        # We could convert the values it into lists, but this works equally well.
        tradeable = np.vectorize(lambda condition_list: "R" in condition_list if type(condition_list) is str else False)

        minute_df.ffill(inplace=True)
        minute_df["tradeable"] = tradeable(minute_df["conditions"])
        minute_df.drop(columns=["conditions"], inplace=True)

        # If there were no quotes for the first minutes of the day (very unlikely), we will forward fill them with the values of the previous day. 
        # We will do that later. 'tradeable' is already false in this case because there was no 'R' in the conditions.

        # Step 3: Get the high and low bid/asks. If the value is empty, it means that during the entire minute there were no new quotes. 
        # Then we must use the last quotes. Since we already filled the close_bid and close_ask with the latest quote, we can set it equal to that.
        high_df = tick_day_df.resample("1Min").max()[["bid_price", "ask_price"]]
        high_df.rename(
            columns={
                "bid_price": "high_bid",
                "ask_price": "high_ask",
            },
            inplace=True,
        )

        low_df = tick_day_df.resample("1Min").min()[["bid_price", "ask_price"]]
        low_df.rename(
            columns={
                "bid_price": "low_bid",
                "ask_price": "low_ask",
            },
            inplace=True,
        )
        minute_df = pd.merge(
            left=minute_df, right=low_df, how="left", left_index=True, right_index=True
        )
        minute_df = pd.merge(
            left=minute_df, right=high_df, how="left", left_index=True, right_index=True
        )

        # Fill high/low with close
        minute_df["low_bid"] = minute_df["low_bid"].fillna(minute_df["close_bid"])
        minute_df["low_ask"] = minute_df["low_ask"].fillna(minute_df["close_ask"])
        minute_df["high_bid"] = minute_df["high_bid"].fillna(minute_df["close_bid"])
        minute_df["high_ask"] = minute_df["high_ask"].fillna(minute_df["close_ask"])

        # Reorder columns
        minute_df = minute_df[
                [
                    "close_bid_size",
                    "close_bid",
                    "close_ask",
                    "close_ask_size",
                    "high_bid",
                    "high_ask",
                    "low_bid",
                    "low_ask",
                    "tradeable",
                ]
            ]

        # Append the df (which only contains the minutes of 1 day) to quote_dfs.
        quote_dfs.append(minute_df) #quote_dfs[0] contains the DataFrame from the first day; quote_dfs[1] the second day etc.

        print(f"Processed {stock} tick data for {date.strftime('%Y-%m-%d')}")
    # Step 4: Concatenate daily DataFrames
    quote_df = pd.concat(quote_dfs)

    # As said in step 2, if there were no quotes for the first minutes of the day (very unlikely), 
    # we will forward fill them with the values of the previous day. 'tradeable' is already False. 
    if quote_df.isnull().any().any():
        amount_null = quote_df.isna().sum().sum()
        print(
            f"{stock} | WARNING: dataframe contain {amount_null} null values, which will be forward filled."
        )
        quote_df = quote_df.ffill().bfill() #Very rarely it can happen that at the start there are no quotes. That is why we also need to backfill. 

    quote_df.to_csv(f"../data/alpaca/processed/m1/quotes/{stock}.csv")
    print(f"{datetime.utcnow().replace(microsecond=0)} | {stock} | Processed tick data to quotes")

**Step 3: Combine quotes and bars to get quotebars.**

* Step 1: Left merge quotes to bars.
* Step 2: Adjust quotes using the adjustment factor and save.

In [ ]:
# SYMBOL_LIST = ["TOP", "APE", "DEM"]
# START_DATE = datetime(2022, 1, 1) #datetime(2015, 12, 1) is the first available date from Alpaca
# END_DATE = datetime(2023, 7, 21)
MARKET_HOURS_ONLY = True

In [ ]:
# for stock in SYMBOL_LIST:
for stock in SYMBOL_LIST:
    quotes = pd.read_csv(
        f"../data/alpaca/processed/m1/quotes/{stock}.csv",
        index_col="datetime",
        parse_dates=True,
    )
    bars = pd.read_csv(
        f"../data/alpaca/processed/m1/bars/{stock}.csv",
        index_col="datetime",
        parse_dates=True,
    )
    quotes = quotes[
            (quotes.index >= START_DATE.replace(hour=4, minute=0, second=0))
            & (quotes.index <= END_DATE.replace(hour=19, minute=59, second=0))
        ]
    bars = bars[
            (bars.index >= START_DATE.replace(hour=4, minute=0, second=0))
            & (bars.index <= END_DATE.replace(hour=19, minute=59, second=0))
        ]
    if MARKET_HOURS_ONLY == True:
        quotes = quotes.between_time("9:30", "15:59")
        bars = bars.between_time("9:30", "15:59")

    # The "tradable" from the quotes is leading. The "tradable" from quotes was just approximated by setting "tradable" 
    # to True if the data was empty. The "tradable" from quotes are whether the stock was actually tradable.
    quotebars = pd.merge(
        left=bars[["open", "high", "low", "close", "close_original", "volume", "adjustment"]],
        right=quotes[["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask", "tradeable"]],
        how="left",
        left_index=True,
        right_index=True,
    )
    quotebars = quotebars[["open", "high", "low", "close", "close_original", "close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask", "volume", "tradeable", "adjustment"]]

    if len(bars) != len(quotes):
        print(f"{stock} | There is a difference between the amount of bars and quotes.")
        print(f"There are {len(bars) - len(quotes)} less quotes than bars")
        # If there are less quotes than bars, tradable will be set to 'False'. The values of the quotes don't 
        # matter at this point (because it will never be used if 'tradable' is False). Hence we will just fill 
        # it with zeroes. This is only the case with multi-day halts.
        quotebars['tradeable'] = quotebars['tradeable'].fillna(False)
        quotebars[["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask"]] = quotebars[["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask"]].fillna(0.0)
 
    # The quotes are NOT adjusted for dividends and splits. So we need to adjust them such 
    # that ALL values in bars and quotebars are adjusted.
    quotebars[["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask"]] = quotebars[["close_bid_size", "close_bid", "close_ask", "close_ask_size", "high_bid", "high_ask", "low_bid", "low_ask"]].multiply(quotebars["adjustment"], axis="index").round(decimals=5) #Rounding to save space, but may cause small precision errors

    quotebars.to_csv(f"../data/alpaca/processed/m1/quotebars/{stock}.csv")
    print(f'Processed {stock} to quotebars')


In [50]:
example = pd.read_csv( 
                f"../data/alpaca/processed/m1/quotebars/DEM.csv",
                index_col="datetime",
                parse_dates=True,
                nrows=5
            )
example

,open,high,low,close,close_original,close_bid_size,close_bid,close_ask,close_ask_size,high_bid,high_ask,low_bid,low_ask,volume,tradeable,adjustment
datetime,,,,,,,,,,,,,,,,
2022-01-03 09:30:00,38.97,38.97,38.97,38.97,43.372093,26.95512,38.92319,39.07594,26.95512,38.94116,39.12985,38.86030,39.06696,2634.0,True,0.898504
2022-01-03 09:31:00,38.97,38.97,38.97,38.97,43.372093,0.89850,38.86030,39.17478,8.98504,38.92319,39.17478,38.86030,39.05797,0.0,True,0.898504
2022-01-03 09:32:00,38.97,38.97,38.97,38.97,43.372093,26.05662,38.88725,39.04000,26.05662,38.89624,39.17478,38.86030,39.02203,0.0,True,0.898504
2022-01-03 09:33:00,39.01,39.01,39.01,39.01,43.416611,27.85362,38.88725,39.04000,29.65063,38.90522,39.04899,38.86030,39.04000,369.0,True,0.898504
2022-01-03 09:34:00,39.01,39.01,39.01,39.01,43.416611,26.95512,38.89624,39.02203,5.39102,38.91421,39.04000,38.87827,39.02203,0.0,True,0.898504


**Step 4: Check for weird quotes**

If the quotebar is tradeable:
* The OHLC should be within or equal to the quotes
* The quotes should not deviate too much from OHLC. I choose a maximum acceptable deviation of 5%.

Since we only have IEX data and I deliberately choose less liquid stocks to decrease downloading time, there will be a lot of weird quotes. Because I won't be using this for real trading, I will not 'clean' this data. If we were to have NBBO data from all exchanges, the amount of weird quotes would go down significantly. I probably wouldn't even have to clean it anymore. Besides, the backtester should handle situations in which the asset is tradeable but has an unacceptably high spread.

In [33]:
# SYMBOL_LIST = ["TOP", "APE", "DEM"]

In [39]:
for stock in SYMBOL_LIST:
    quotebars = pd.read_csv(
            f"../data/alpaca/processed/m1/quotebars/{stock}.csv",
            index_col="datetime",
            parse_dates=True,
        )
    # ['open', 'high', 'low', 'close', 'close_original', 'close_bid_size',
    #        'close_bid', 'close_ask', 'close_ask_size', 'high_bid', 'high_ask',
    #        'low_bid', 'low_ask', 'volume', 'tradeable', 'adjustment'],

    quotebars = quotebars[quotebars['tradeable']==True]
    rows = len(quotebars)
    # If there were no trades, close is forward filled in the OHLC bars
    # It can then easily happen that is it not contained between bid/ask
    # Because bid/ask can change without there being a trade
    quotebars = quotebars[quotebars['volume'] > 0]

    # 1. Check percentage of quotebars where OHLC is outside quotes
    # The extra 0.01 tolerance if for rounding issues due to adjustments
    quotebars = quotebars[['open', 'high', 'low', 'close', 'close_bid_size','close_bid', 'close_ask', 'close_ask_size','high_bid', 'high_ask','low_bid', 'low_ask', 'volume']]
    not_contained_close = quotebars[(quotebars['close'] < quotebars['close_bid']-0.01) | ((quotebars['close'] > quotebars['close_ask']+0.01))]
    not_contained_high = quotebars[(quotebars['high'] > quotebars['high_ask']+0.01)]
    not_contained_low = quotebars[(quotebars['low'] < quotebars['low_bid']-0.01)]

    not_contained_close = not_contained_close[['close', 'close_bid', 'close_ask', 'volume']]
    not_contained_high = not_contained_high[['high', 'high_ask', 'volume']]
    not_contained_low = not_contained_low[['low', 'low_bid', 'volume']]

    print(f"{stock} | TRADES NOT CONTAINED IN QUOTES")
    print(f"    close: {100*(len(not_contained_close)/rows):.2f}% is not correct")
    print(f"    high: {100*(len(not_contained_high)/rows):.2f}% is not correct")
    print(f"    low: {100*(len(not_contained_low)/rows):.2f}% is not correct")

    # 2. Check if quotes are not too far from OHLC
    too_far_close = quotebars[(quotebars['close']*1.05 < quotebars['close_bid']) | ((quotebars['close']*0.95 > quotebars['close_ask']))]
    too_far_high = quotebars[(quotebars['high_ask'] > quotebars['high']*1.05)]
    too_far_low = quotebars[(quotebars['low_bid'] < quotebars['low']*0.95)]

    too_far_close = too_far_close[['close', 'close_bid', 'close_ask', 'volume']]
    too_far_high = too_far_high[['high', 'high_ask', 'volume']]
    too_far_low = too_far_low[['low', 'low_bid', 'volume']]

    print(f"{stock} | QUOTES DEVIATING >5% FROM TRADES")
    print(f"    close: {100*(len(too_far_close)/rows):.2f}% is not correct")
    print(f"    high: {100*(len(too_far_high)/rows):.2f}% is not correct")
    print(f"    low: {100*(len(too_far_low)/rows):.2f}% is not correct")
    print(f"=============================================================")


TOP | TRADES NOT CONTAINED IN QUOTES
    close: 1.56% is not correct
    high: 0.19% is not correct
    low: 0.20% is not correct
TOP | QUOTES DEVIATING >5% FROM TRADES
    close: 0.00% is not correct
    high: 0.44% is not correct
    low: 0.28% is not correct
APE | TRADES NOT CONTAINED IN QUOTES
    close: 0.02% is not correct
    high: 0.05% is not correct
    low: 0.04% is not correct
APE | QUOTES DEVIATING >5% FROM TRADES
    close: 0.00% is not correct
    high: 0.03% is not correct
    low: 0.03% is not correct
DEM | TRADES NOT CONTAINED IN QUOTES
    close: 2.64% is not correct
    high: 0.03% is not correct
    low: 0.03% is not correct
DEM | QUOTES DEVIATING >5% FROM TRADES
    close: 0.00% is not correct
    high: 0.00% is not correct
    low: 0.00% is not correct


**Step 5: Delete all tick data**

We do not need the tick data anymore and it takes up an enormous amount of disk space. For reference, the ['Complete Tick Data'](https://firstratedata.com/tick-data) bundle at [firstratedata.com](https://firstratedata.com/) which contains trades and quotes of 5000 stocks & ETFs for almost 14 years is 24 TB. I only have 2 disks of 0.25TB... I ordered a 4TB disk, I hope that is more than enough.

*I eventually decided to just delete manually because that is easier.*